# 学习目标：
- 掌握Qwen模型基本特性
- 掌握Baichuan-7B基本特性
- 掌握NLP四种范式，并知道

# 1、Qwen模型
概述：Qwen大语言模型是是有阿里巴巴训练并开源的一系列大语言模型，最早开始2023年8月开源70亿参数规模，随后几个月开始陆续开源其他模型，最低参数18亿，最高参数720亿

训练目标：在训练目标上，Qwen的核心训练目标是语言模型，即根据已有的上下文去预测下一个词
模型结构：
- 分组查询注意力：通过减少键值（KV）缓存的作用，提升了模型的推理效率
- 激活函数：使用SwiGLU激活函数，增强了模型的非线性表达能力
- 位置编码：采用了旋转编码RoPE，提升了模型对长序列的处理能力
- 层归一化（Layer Normalization）：使用了RMSNorm，确保了训练的稳定性

模型配置：

| 配置       | 数据                    |
| ---------- | ----------------------- |
| 参数       | 7B                      |
| 隐藏层维度 | 3584                    |
| 层数       | 28                      |
| 注意力头数 | 28个query、4个key-value |
| 训练数据   | 1.2T                    |
| 词表大小   | 151936                  |
| 最大长度   | 32768                   |


大规模叶柄，日本在美国

# 2、Baichuan-7B模型
概述：Baichuan-7B是由百川智能于2023年6月发布的一个开发且可商用的大型预训练，支持中英文双语，是在约1.2万亿哥token上训练70亿参数模型

训练目标：
- Baichuan-7B的训练目标也是语言模型，即根据已有的上文去预测下一个词
- 关于tokenizer使用了BPE分词算法作为tokenizer，词表大小64000
- 关于数据，原始数据包括开源的中英文数据和自行抓取的中文互联网数据，以及部分高质量知识性数据

模型结构：
- 和LLaMA一样的模型设计，也是Decoder-Only架构，但是结合前人的工作做了一些改进
    - Pre-Normalization：为了提高训练的稳定性，没有使用传统的Post-Layer-Norm 而是使用了Pre-Layer-Norm并同时使用RMSNorm归一化函数（RMS Norm的主要区别在于去掉了减去均值部分，简化了LayerNorm的计算，可以减少约7%～64%计算时间）
    - 激活函数：使用SwiGLU激活函数
    - 位置编码：采用旋转位置编码RoPE

模型配置：

| 配置       | 数据    |
|:---------|:------|
| 参数       | 7B    |
| 隐藏层维度    | 4096  |
| 层数       | 32    |
| 注意力头数    | 32    |
| 训练数据     | 1.2T  |
| 词表大小     | 64000 |
| 最大长度     | 4096  |



# 1、NLP四种范式分别是？
- 第一范式：基于“传统机器学习模型”的范式
- 第二范式：基于“深度学习模型”的范式
- 第三范式：基于“预训练模型+Fine+Tuning”的范式
- 第四范式：基于“预训练模型+Prompt+预测”的范式


# Prompt-Tuning（提示微调）
Prompt-Tuning执行步骤：
- 构建模版（Template Construction）：通过人工定义、自动搜索、文本生成等方法，生成与给定句子相关的一个含有[MASK]标记的模板。例如It was [MASK].，并拼接到原始的文本中，获得Prompt-Tuning的输入：[CLS] I like the Disney films very much. [SEP] It was [MASK]. [SEP]。将其喂入BERT模型中，并复用预训练好的MLM分类器（在huggingface中为BertForMaskedLM），即可直接得到[MASK]预测的各个token的概率分布。
- 便签次映射（Label Word Verbalizer）：因为[MASK]部分我们只对部分词感兴趣，因此需要建立一个映射关系。例如如果[MASK]预测的词是“great”，则认为是positive类，如果是“terrible”，则认为是negative类。
- 训练：根据Verbalizer，则可以获得指定label word的预测概率分布，并采用交叉信息熵进行训练。此时因为只对预训练好的MLM head进行微调，所以避免了过拟合问题。


# PET模型
PET模型提出来两个很重要的组建：
- Pattern（Template）：记作T, 即上文提到的Template，其为额外添加的带有[mask]标记的短文本，通常一个样本只有一个Pattern（因为我们希望只有1个让模型预测的[mask]标记）。由于不同的任务、不同的样本可能会有其更加合适的pattern，因此如何构建合适的pattern是Prompt-Tuning的研究点之一；
- Verbalizer：记作V, 即标签词的映射，对于具体的分类任务，需要选择指定的标签词（label word）。例如情感分析中，我们期望Verbalizer可能是 （positive和negative是类标签）。同样，不同的任务有其相应的label word，但需要注意的是，Verbalizer的构建需要取决于对应的Pattern。因此 如何构建Verbalizer是另一个研究挑战 。

# Hard Prompt & Soft Prompt
- Hard Prompt (离散提示)：是一种固定的提示模板，通过将特定的关键词或短语(真实的文本字符串)直接嵌入到文本中，引导模型生成符合要求的文本。这种提示方法的特点在于，提示模板是固定的，不能根据不同的任务和需求进行调整。
- Soft Prompt (连续提示) ：是指通过给模型输入一个可参数化的提示模板，从而引导模型生成符合特定要求的文本。这种提示方法的特点在于，提示模板中的参数可以根据具体任务和需求进行调整，以达到最佳的生成效果。

# P-tuning（NLU任务）

